In [1]:
!pip install fastcoref

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 82.3 MB/s eta 0:00:00:00:0100:01
  Created wheel for fastcoref: filename=fastcoref-2.1.6-py3-none-any.whl size=31253 sha256=f78379fa45dfb914c870e742c061c5ca9add83518aa03c2ec80d4328d99c55ba
  Stored in directory: /root/.cache/pip/wheels/73/ac/49/24d3e434b85513f46503fbea7e7eb6353e32b66619ea86b410
Successfully built fastcoref
  Attempting uninstall: numpy
    Found existing installation: numpy 1.26.4
    Uninstalling numpy-1.26.4:
      Successfully uninstalled numpy-1.26.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.10.1 requires cubinlinker, which is not installed.
cudf 24.10.1 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 24.10.1 requires libcudf==24.10.*, whic

In [2]:
# from fastcoref import FCoref

# # Khởi tạo mô hình
# model = FCoref(device='cuda:0')  # Chọn GPU nếu có, nếu không thì để 'cpu'

# # Văn bản đầu vào
# texts = [
#     "Alice goes down the rabbit hole. She finds a strange world.",
#     "Bob loves programming. He codes every day."
# ]

# # Dự đoán
# preds = model.predict(texts=texts)

# # Lấy các cụm từ có liên quan (clusters)
# for i, pred in enumerate(preds):
#     print(f"Text {i + 1}:")
#     print("Clusters:", pred.get_clusters())
#     print()


config.json:   0%|          | 0.00/819 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/393 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/362M [00:00<?, ?B/s]

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Inference:   0%|          | 0/2 [00:00<?, ?it/s]

Text 1:
Clusters: []

Text 2:
Clusters: [['Bob', 'He']]



Tích hợp với SpaCy
Bạn có thể tích hợp fastcoref như một pipeline trong SpaCy:

In [23]:
from fastcoref import spacy_component
import spacy

# Tải SpaCy và thêm pipeline `fastcoref`
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("fastcoref")

# Dự đoán với văn bản
doc = nlp("Alice goes down the rabbit hole. She finds a strange world. Bob loves programming. He codes every day.")
print(doc._.coref_clusters)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[[(0, 5), (33, 36)], [(60, 63), (83, 85)]]


In [24]:
# Hiển thị các cụm từ tham chiếu
for cluster in doc._.coref_clusters:
    resolved_text = [doc.text[start:end] for start, end in cluster]
    print("Cluster:", resolved_text)


Cluster: ['Alice', 'She']
Cluster: ['Bob', 'He']


In [32]:
# Văn bản gốc
text = "Alice goes down the rabbit hole. She finds a strange world. Bob loves programming. He codes every day."

# Dự đoán với văn bản
doc = nlp(text)

# Hiển thị các cụm từ tham chiếu
for cluster in doc._.coref_clusters:
    resolved_mentions = [text[start:end] for start, end in cluster]
    print("Cluster:", resolved_mentions)

# Chuyển văn bản thành danh sách ký tự để tránh lỗi chỉ số
tokens = list(text)

# Thay thế đại từ nhân xưng bằng thực thể chính
for cluster in doc._.coref_clusters:
    main_mention = text[cluster[0][0]:cluster[0][1]]  # Cụm chính
    for start, end in cluster[1:]:  # Các cụm còn lại
        pronoun = text[start:end]
        # Chỉ thay thế nếu là đại từ nhân xưng
        if pronoun.lower() in ["he", "she", "him", "her", "his", "hers", "they", "them", "their"]:
            # Thay thế trực tiếp trên danh sách ký tự
            tokens[start:end] = list(main_mention)

# Ghép danh sách ký tự thành chuỗi hoàn chỉnh
resolved_text = ''.join(tokens)

print("\nOriginal Text:")
print(text)
print("\nResolved Text (Pronouns Replaced):")
print(resolved_text)


Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

Cluster: ['Alice', 'She']
Cluster: ['Bob', 'He']

Original Text:
Alice goes down the rabbit hole. She finds a strange world. Bob loves programming. He codes every day.

Resolved Text (Pronouns Replaced):
Alice goes down the rabbit hole. Alice finds a strange world. Bob loves programmingBobHe codes every day.


### **Hướng dẫn và giải thích repo `fastcoref`**

`fastcoref` là một thư viện Python mạnh mẽ để thực hiện nhiệm vụ "Coreference Resolution" - xác định các cụm từ hoặc từ nào trong văn bản tham chiếu đến cùng một thực thể.

Dưới đây là từng bước giải thích nội dung của repository này và hướng dẫn cài đặt và sử dụng:

---

### **1. Cài đặt**

Bạn cần cài đặt thư viện này thông qua `pip`:

```bash
pip install fastcoref
```

Nếu bạn muốn huấn luyện mô hình riêng, hãy cài đặt thêm các phụ thuộc huấn luyện:

```bash
pip install fastcoref[train]
```

---

### **2. Chức năng chính: Coreference Resolution**

Coreference Resolution là nhiệm vụ xác định các từ hoặc cụm từ nào trong văn bản tham chiếu đến cùng một thực thể. Ví dụ:

- Văn bản: `"Alice goes down the rabbit hole. She finds a strange world."`
- Kết quả: `"She"` tham chiếu đến `"Alice"`.

---

### **3. Sử dụng cơ bản**

Dưới đây là cách sử dụng `fastcoref` để thực hiện coreference resolution:

#### **Code cơ bản**
```python
from fastcoref import FCoref

# Khởi tạo mô hình
model = FCoref(device='cuda:0')  # Chọn GPU nếu có, nếu không thì để 'cpu'

# Văn bản đầu vào
texts = [
    "Alice goes down the rabbit hole. She finds a strange world.",
    "Bob loves programming. He codes every day."
]

# Dự đoán
preds = model.predict(texts=texts)

# Lấy các cụm từ có liên quan (clusters)
for i, pred in enumerate(preds):
    print(f"Text {i + 1}:")
    print("Clusters:", pred.get_clusters())
    print()
```

#### **Kết quả**
```plaintext
Text 1:
Clusters: [['Alice', 'She']]

Text 2:
Clusters: [['Bob', 'He']]
```

---

### **4. Giải thích từng thành phần**

#### **`model.predict`**
- Nhận danh sách các văn bản.
- Trả về danh sách `CorefResult` cho từng văn bản.

#### **Các phương thức trong `CorefResult`**
1. **`get_clusters()`**:
   - Trả về danh sách các cụm từ tham chiếu đến cùng thực thể.

2. **`get_clusters(as_strings=False)`**:
   - Trả về danh sách các chỉ số (start, end) của cụm từ trong văn bản gốc.

3. **`get_logit(span_i, span_j)`**:
   - Trả về điểm xác suất rằng hai cụm từ `span_i` và `span_j` tham chiếu đến cùng thực thể.

---

### **5. Sử dụng với văn bản đã tokenized**

Nếu văn bản đã được tokenized (chia từ sẵn), bạn có thể đặt `is_split_into_words=True`:

#### **Code**
```python
preds = model.predict(
    texts=[["Alice", "goes", "down", "the", "rabbit", "hole", ".", "She", "finds", "a", "strange", "world", "."]],
    is_split_into_words=True
)
print(preds[0].get_clusters())
```

#### **Kết quả**
```plaintext
[['Alice', 'She']]
```

---

### **6. Tăng tốc với xử lý hàng loạt**

Để xử lý nhiều văn bản cùng lúc và tối ưu hóa tài nguyên, bạn có thể sử dụng tham số `max_tokens_in_batch`:

#### **Code**
```python
texts = ["Alice goes to school.", "Bob plays soccer."] * 1000

# Sử dụng batch processing
preds = model.predict(texts=texts, max_tokens_in_batch=100)
```

---

### **7. Tích hợp với SpaCy**

Bạn có thể tích hợp `fastcoref` như một pipeline trong SpaCy:

#### **Code**
```python
from fastcoref import spacy_component
import spacy

# Tải SpaCy và thêm pipeline `fastcoref`
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("fastcoref")

# Dự đoán với văn bản
doc = nlp("Alice goes down the rabbit hole. She finds a strange world.")
print(doc._.coref_clusters)
```

#### **Kết quả**
```plaintext
[[('Alice', 0, 5), ('She', 27, 30)]]
```

---

### **8. Huấn luyện mô hình riêng**

Nếu bạn muốn huấn luyện mô hình riêng, thư viện hỗ trợ:

1. **Chuẩn bị dữ liệu:**
   - Dữ liệu cần có định dạng `jsonlines`, với thông tin cụm từ (clusters).

2. **Sử dụng `CorefTrainer` để huấn luyện:**
   ```python
   from fastcoref import TrainingArgs, CorefTrainer

   args = TrainingArgs(
       output_dir='my_model',
       model_name_or_path='distilroberta-base',
       device='cuda:0',
       epochs=10
   )

   trainer = CorefTrainer(
       args=args,
       train_file='train_data.jsonlines',
       dev_file='dev_data.jsonlines'
   )
   trainer.train()
   ```

3. **Tải mô hình đã huấn luyện:**
   ```python
   from fastcoref import FCoref
   model = FCoref(model_name_or_path='my_model', device='cuda:0')
   ```

---

### **Tóm tắt**

- **Cài đặt:** `pip install fastcoref`
- **Dự đoán:** `model.predict()`
- **Tích hợp SpaCy:** `nlp.add_pipe("fastcoref")`
- **Huấn luyện mô hình riêng:** Sử dụng `CorefTrainer`.

Nếu bạn cần hỗ trợ thêm, hãy cho biết chi tiết!

### **Giải thích kết quả `doc._.coref_clusters`**

Khi bạn chạy đoạn mã:

```python
from fastcoref import spacy_component
import spacy

# Tải SpaCy và thêm pipeline `fastcoref`
nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("fastcoref")

# Dự đoán với văn bản
doc = nlp("Alice goes down the rabbit hole. She finds a strange world. Bob loves programming. He codes every day.")
print(doc._.coref_clusters)
```

#### **Kết quả:**
```plaintext
[[(0, 5), (33, 36)], [(60, 63), (83, 85)]]
```

---

### **Giải thích từng phần**

1. **Kết quả dạng danh sách các cụm (clusters):**

   - `doc._.coref_clusters` trả về danh sách các cụm từ mà mô hình nhận diện là tham chiếu đến cùng một thực thể.

2. **Cấu trúc kết quả:**
   ```python
   [[(0, 5), (33, 36)], [(60, 63), (83, 85)]]
   ```
   - Mỗi cụm (cluster) là một danh sách các cặp `(start, end)`:
     - `(start, end)` đại diện cho vị trí ký tự bắt đầu và kết thúc của một cụm từ trong văn bản gốc.

---

### **Ví dụ chi tiết với văn bản:**

```plaintext
Alice goes down the rabbit hole. She finds a strange world. 
Bob loves programming. He codes every day.
```

#### **Kết quả phân tích cụm:**
1. **Cụm đầu tiên: `[(0, 5), (33, 36)]`**
   - `(0, 5)` → `"Alice"` (từ ký tự 0 đến 5 trong văn bản).
   - `(33, 36)` → `"She"` (từ ký tự 33 đến 36 trong văn bản).
   - **Kết quả:** `"Alice"` và `"She"` tham chiếu đến cùng một thực thể.

2. **Cụm thứ hai: `[(60, 63), (83, 85)]`**
   - `(60, 63)` → `"Bob"` (từ ký tự 60 đến 63 trong văn bản).
   - `(83, 85)` → `"He"` (từ ký tự 83 đến 85 trong văn bản).
   - **Kết quả:** `"Bob"` và `"He"` tham chiếu đến cùng một thực thể.

---

### **Hướng dẫn thêm để trực quan hóa:**

Bạn có thể sử dụng Python để truy xuất trực tiếp các từ hoặc cụm từ từ văn bản:

```python
# Hiển thị các cụm từ tham chiếu
for cluster in doc._.coref_clusters:
    resolved_text = [doc.text[start:end] for start, end in cluster]
    print("Cluster:", resolved_text)
```

#### **Kết quả:**
```plaintext
Cluster: ['Alice', 'She']
Cluster: ['Bob', 'He']
```

---

### **Tóm tắt:**
- **`doc._.coref_clusters`** trả về danh sách các cụm tham chiếu trong văn bản.
- **Mỗi cụm** gồm các cặp `(start, end)` biểu diễn vị trí trong văn bản gốc.
- **Cách sử dụng:** Có thể dùng các chỉ số `(start, end)` để trích xuất các cụm từ gốc hoặc dùng trực tiếp cho các ứng dụng NLP.

## Load Dataset 


In [10]:
import json

# Đường dẫn tới file JSON
file_path = '/kaggle/input/rag-dataset/data/corpus/raw/hotpotqa.json'


with open(file_path, 'r', encoding='utf-8') as file:
    data = json.load(file)

In [16]:
data[0]['paragraph_text']

'Passage 1:\nTrusty system (prison)\nThe "trusty system" (sometimes incorrectly called "trustee system") was a penitentiary system of discipline and security enforced in parts of the United States until the 1980s, in which designated inmates were given various privileges, abilities, and responsibilities not available to all inmates.It was made compulsory under Mississippi state law but was used in other states as well, such as Arkansas, Alabama, Louisiana, New York and Texas. The method of controlling and working inmates at Mississippi State Penitentiary at Parchman was designed in 1901 to replace convict leasing. The case Gates v. Collier ended the flagrant abuse of inmates under the trusty system and other prison abuses that had continued essentially unchanged since the building of the Mississippi State Penitentiary. Other states using the trusty system were also forced to give it up under the ruling.\n\nHistory\nPrisons had trusties as far back as the 1800s.\n\nParchman Farm\nThe pr

In [17]:
# from fastcoref import spacy_component
# import spacy

# # Tải SpaCy và thêm pipeline `fastcoref`
# nlp = spacy.load("en_core_web_sm")
# nlp.add_pipe("fastcoref")

# Dự đoán với văn bản
text = data[0]['paragraph_text']
doc = nlp(text)
print(doc._.coref_clusters)


# Hiển thị các cụm từ tham chiếu
for cluster in doc._.coref_clusters:
    resolved_text = [doc.text[start:end] for start, end in cluster]
    print("Cluster:", resolved_text)



Map:   0%|          | 0/1 [00:00<?, ? examples/s]

Inference:   0%|          | 0/1 [00:00<?, ?it/s]

[[(34, 101), (331, 333), (687, 704), (847, 864), (890, 892), (2601, 2618), (3031, 3048), (3987, 4004), (4024, 4041), (4119, 4121), (4413, 4430)], [(527, 569), (792, 826), (985, 995), (1154, 1164), (2018, 2030), (3136, 3146)], [(1460, 1475), (1477, 1481), (1572, 1591)], [(1720, 1779), (1781, 1786)], [(2620, 2627), (2681, 2686)], [(2763, 2781), (2793, 2796)], [(3177, 3204), (3275, 3277)], [(2923, 2988), (3524, 3526), (4132, 4148)], [(428, 436), (4051, 4059)], [(438, 445), (4061, 4068)], [(447, 456), (4070, 4079)], [(471, 476), (4085, 4090), (4188, 4193)], [(4167, 4194), (4211, 4216)], [(200, 209), (4275, 4284)]]
Cluster: ['The "trusty system" (sometimes incorrectly called "trustee system")', 'It', 'the trusty system', 'the trusty system', 'it', 'the trusty system', 'the trusty system', 'the trusty system', 'the trusty system', 'it', 'the trusty system']
Cluster: ['Mississippi State Penitentiary at Parchman', 'the Mississippi State Penitentiary', 'The prison', 'the prison', "the prison's"